In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import json
import pandas as pd
import os
import sys
import cv2
sys.path.append('..')
import difflib
from fidelity_check import find_diff_writes

In [4]:
screenshot_similarity = json.load(open("../record_replay/screenshots/onload_screenshot_similarity.json", 'r'))
screenshot_similarity = {s['directory']: s['screenshot_similarity'] for s in screenshot_similarity for s in screenshot_similarity}

In [6]:
def check_fidelity(dirr):
    screenshot_simi = screenshot_similarity[dirr.split('/')[-1]]
    live_html = open(f"{dirr}/live.html").read()
    archive_html = open(f"{dirr}/archive.html").read()
    live_element = json.load(open(f"{dirr}/live_elements.json"))
    archive_element = json.load(open(f"{dirr}/archive_elements.json"))
    my_fidelity = fidelity_check.verify(live_html, live_element, archive_html, archive_element)
    print(screenshot_simi >= 1, my_fidelity)

# check_fidelity('testcases/theftaz.azag.gov_1')

#### Test difflib

In [27]:
e1 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e2 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e1 == e2

/a/b /a/b True


True

## Check archive differences

In [10]:
dirr = 'testcases/www.slideshare.net_1'
dirr = '../execution_match/examples/eta.lbl.gov_2'

live_element = json.load(open(f"{dirr}/live_elements.json"))
archive_element = json.load(open(f"{dirr}/archive_elements.json"))
# fidelity_check.verify( live_element, archive_element)
live_unique, archive_unique = fidelity_check.diff(live_element, archive_element, returnHTML=False)
print(len(live_unique), len(archive_unique))
for i in live_unique:
    print("live:\n", i)
for i in archive_unique:
    print("archive:\n", i)

live_unique number [2, 1, 1, 1, 1, 1, 1, 1]
archive_unique number [2, 1, 1, 1, 1, 1, 7, 1]
8 8
live:
 ['/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]', '/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/picture[1]/img[1]']
live:
 ['/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[5]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/iframe[1]/html[1]/body[1]/div[1]/div[1]/a[1]/div[1]/div[2]/svg[1]/path[1]']
live:
 ['/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[5]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/iframe[1]/html[1]/body[1]/div[1]/div[1]/a[1]/div[1]/div[2]/svg[1]/use[1]']
live:
 ['/html[1]/body[1]/div[2]/div[2]/section[1]/div[3]/div[3]/article[1]/div[1]/div[1]/div[5]/div[1]/div[1]/div[2]/div[1]/div[1]/div[1]/iframe[1]/html[1]/body[1]/div[1]/div[1]/div[3]/div[1]/div[1]']
live:
 ['/html[1]/body[1

In [99]:
# * Generate testcase
diff = {"live": [], "archive": []}
for lu in live_unique:
    first_line = lu.strip().split('\n')[0]
    diff['live'].append({"root": first_line, "numlines": len(lu.strip().split('\n'))})
for au in archive_unique:
    first_line = au.strip().split('\n')[0]
    diff['archive'].append({"root": first_line, "numlines": len(au.strip().split('\n'))})
print(json.dumps(diff, indent=4))

{
    "live": [
        {
            "root": "<div class=\"media media--blazy media--bundle--slideshow-carousel media--slick media--responsive media--image is-b-loaded\" style=\"\" tabindex=\"0\">",
            "numlines": 2
        },
        {
            "root": "<div class=\"ytp-title-channel\">",
            "numlines": 1
        },
        {
            "root": "<div class=\"ytp-title-channel\">",
            "numlines": 1
        }
    ],
    "archive": [
        {
            "root": "<div class=\"media media--blazy media--bundle--slideshow-carousel media--slick media--loading is-b-loading media--responsive media--image\" style=\"\" tabindex=\"0\">",
            "numlines": 2
        },
        {
            "root": "<button class=\"ytp-watch-later-button ytp-button ytp-show-watch-later-title\" data-tooltip-opaque=\"true\" title=\"Watch later\" aria-label=\"Watch later\">",
            "numlines": 7
        }
    ]
}


### Check the assumption of archive's writes are always a subset of live

In [6]:
dirs = os.listdir('../record_replay/writes')
num_diff, num_diff_writes = 0, 0
total = 0
for dirr in dirs:
    if not os.path.exists(f'../record_replay/writes/{dirr}/live_elements.json') or not os.path.exists(f'../record_replay/writes/{dirr}/archive_elements.json'):
        continue
    key_writes = find_diff_writes.locate_key_writes(f'../record_replay/writes/{dirr}')
    live_additional = [len(w['key_related_writes']) for w in key_writes['live']]
    archive_additional = [len(w['key_related_writes']) for w in key_writes['archive']]

    total += 1
    if len(live_additional) > 0 or len(archive_additional) > 0:
        num_diff += 1
    # if sum(archive_additional) > 0:
    #     num_diff_writes += 1

    ive_unique_elements, archive_unique_elements = find_diff_writes.find_diff_elements(f'../record_replay/writes/{dirr}')
    diff_writes = find_diff_writes.collect_diff_writes(f'../record_replay/writes/{dirr}')
    if len(diff_writes['archive']) > 0:
        print(len(diff_writes['archive']), len(archive_unique_elements))
        num_diff_writes += 1
print("Total", total)
print("Number of different layout trees", num_diff)
print("Number of archive's additional writes", num_diff_writes)

906 1
528 3
2 3
2 0
2 0
2 0
3 0
27 0
1 0
1 0
4 0
1 0
1 0
4 6
1 1
24 72
3 7
1 4
1 0
3 0
4 0
34 7
2 1
1 2
3 0
37 1
7 4
1 1
Total 86
Number of different layout trees 41
Number of archive's additional writes 28


### fidelity check on a set of loads

In [14]:
def valid_dir(dirr):
    valid = os.path.exists(f'{dirr}/live_elements.json') and os.path.exists(f'{dirr}/archive_elements.json')
    return valid

base = '../record_replay/writes'
data = json.load(open(f'{base}/fidelity_check_metadata.json', 'r'))
results = []
for url, info in list(data.items()):
    dirr = info['directory']
    if not valid_dir(f"{base}/{dirr}"):
        continue
    issue, _ =  find_diff_writes.fidelity_issue(f"{base}/{dirr}")
    issue_screenshot, simi = find_diff_writes.fidelity_issue_screenshot(f"{base}/{dirr}")
    results.append({
        'directory': dirr,
        'issue': issue,
        'issue_screenshot': issue_screenshot,
        'screenshot_similarity': simi
    })
json.dump(results, open('fidelity_check.json', 'w+'), indent=2)

In [20]:
dirr = 'its.ntia.gov_1'
issue, diffs =  find_diff_writes.fidelity_issue(f"{base}/{dirr}")
print(issue, json.dumps(diffs, indent=2))

False [
  [],
  []
]
